In [14]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import os
import pandas as pd
import numpy as np

import itertools

import nltk
# nltk.download('stopwords')
# nltk.download('punkt')


# All files are expected to be in same folder
def parse_data(folder_path='anecdots', files_cnt=1):
    parsed_values: list = []
    cnt = 1
    for each in os.listdir(folder_path):
        with open(folder_path + '/' + each, 'r') as f:
            buf = pd.read_csv(folder_path + '/' + each, sep=',')
            parsed_values += buf['content'].tolist()
        if cnt >= files_cnt:
            break
        else:
            cnt += 1
    return parsed_values


class AE(nn.Module):
    def __init__(self, input_shape: int):
        super().__init__()
        # Encoder
        self.line1 = nn.Linear(in_features=input_shape, out_features=input_shape * 3)
        self.line2 = nn.Linear(in_features=input_shape * 3, out_features=input_shape * 9)
        self.line3 = nn.Linear(in_features=input_shape * 9, out_features=input_shape)
        self.line4 = nn.Linear(in_features=input_shape, out_features=input_shape // 4)

        # Decoder
        self.line5 = nn.Linear(in_features=input_shape // 4, out_features=input_shape // 2)
        self.line6 = nn.Linear(in_features=input_shape // 2, out_features=input_shape)
        
        # Weight init
        self.line1.weight.data.normal_(0.0,1/np.sqrt(input_shape))
        self.line2.weight.data.normal_(0.0,1/np.sqrt(input_shape))
        self.line3.weight.data.normal_(0.0,1/np.sqrt(input_shape))
        self.line4.weight.data.normal_(0.0,1/np.sqrt(input_shape))
        self.line5.weight.data.normal_(0.0,1/np.sqrt(input_shape))
        self.line6.weight.data.normal_(0.0,1/np.sqrt(input_shape))

    def forward(self, data: torch.Tensor):
        z = self.encode(data)
        z = self.decode(z)
        return z

    def encode(self, data: torch.Tensor):
        z = F.leaky_relu(self.line1(data))
        z = F.leaky_relu(self.line2(z))
        z = F.leaky_relu(self.line3(z))
        z = F.leaky_relu(self.line4(z))
        return z

    def decode(self, features: torch.Tensor):
        z = F.relu(self.line5(features))
        return self.line6(z)


# data: list of words in 2d
def idx_data(data: list):
    lookup = sorted(list(set(itertools.chain.from_iterable([sentence_data for sentence_data in data]))))
    lookup = {value: index for index, value in enumerate(lookup, 1)}
    return lookup, {index: value for index, value in enumerate(lookup, 1)}


def coalesce(*inputs):
    for i in range(len(inputs)):
        if inputs[i] is not None:
            return inputs[i]
    return 0


# 1D list of sentences
def preprocess(text: list) -> (torch.Tensor, dict, dict):
    # Tokenize all sentences to words. Format is 2D: <sentence, word>
    tokenized_dataset = list()
    for joke in text:
        tokenized_dataset.append(nltk.tokenize.word_tokenize(joke, language='russian'))

    # Drop tail (optional)
    res_len = [len(tokenized_dataset[i]) for i in range(len(tokenized_dataset))]
    tokenized_dataset = tokenized_dataset[:len(tokenized_dataset) - len(tokenized_dataset) % batch_size]

    # Convert tokens to vectors using Word2Vec
    word_to_idx, ids_to_word = idx_data(tokenized_dataset)
    indexes = []
    for sentence in tokenized_dataset:
        indexes.append([coalesce(word_to_idx.get(word)) for word in sentence])

    # Pad to 2D matrix
    max_line_len = len(max(tokenized_dataset, key=len))
    tensor = torch.zeros(size=(len(text), max_line_len))
    for i in range(len(indexes)):
        for j in range(len(indexes[i])):
            tensor[i, j] = indexes[i][j]

    return tensor, word_to_idx, ids_to_word, res_len

### Training
Well, labels are sort-of "how close are we to the source"

In [15]:
# Get some data for model. We have Russian jokes.
batch_size = 32
rus_data = parse_data()

# <cnt of lines, cnt of words>
dataset, direct_lookup, reverse_lookup, batch_lens = preprocess(rus_data)
# print(batch_lens)
dataset /= len(direct_lookup)

# Device
device = torch.device('cpu') if not torch.cuda.is_available() else torch.device('cuda')

# Our model
model = AE(input_shape=dataset.shape[1])
model.to(device)

optimizer = torch.optim.Adam(model.parameters(), lr=.001)
loss = nn.CrossEntropyLoss()
loss.to(device)

# Train
model.train()
prev_sum = .0
for epoch in range(20):
    loss_sum = .0
    for batch in range(len(dataset) // batch_size):
        optimizer.zero_grad()
        output = model(dataset[batch_size*batch:(batch+1)*batch_size, :])
        # Generate labels {as True, False}
        labels = torch.sign(torch.round(torch.abs(torch.sum(output - dataset[batch_size*batch:(batch+1)*batch_size, :], dim=1))))

        loss_res = loss(output, labels.long())
        loss_sum += loss_res
        loss_res.backward()
        optimizer.step()
    print(epoch, loss_sum.item())
    if loss_sum.item() < 1e-6:
        break


0 13.144420623779297
1 0.0


### Now, let's see what we got
Spoiler - schizophrenics will understand

In [28]:
print(f'Length of direct lookup: {len(direct_lookup)}')
model.eval()

# Use data
for i in range(10):
    testing: torch.Tensor = model(dataset[i, :])
    # Yes, we had to use normalization in the end.
    testing -= testing.min()
    values: np.ndarray = np.round(testing.detach().numpy())
    print('> ', rus_data[i], '\n', ' '.join([reverse_lookup.get(idx) for idx in values.tolist()[:batch_lens[2]] if idx in reverse_lookup.keys()]).lower(), end='\n\n')

Length of direct lookup: 48654
>  Читая новости, что Навальный отравился пластиковым стаканчиком, начинаешь понимать, почему их так боится Росгвардия. 
 -сергей али-мпийский -зря -одних -это .потому .испанская .автор -мля -колоссально -божешь -сосед .все-таки -награждением -ихний .- .один

>  Смотрел как Маск запускает космический корабль. Так и не понял куда они прячут попа. Ну не могло же оно взлететь без святой воды. 
 -этому барыг -я .sb -красота 02.02.20 .через .нельзя -о -мы -на -а .пдд -первый -является .есть 0,000000000000000000000000000001

>  - Ну что, отец, печенеги в городе есть?- Кому и депутат печенег. 
 -нет.-вы eddisian -семилетний -у -подсудимый .- -одних -дык -дочка -девушка -почему -нет.-тогда -ихний -здравствуйте -сергей -а -со

>  Если Путин знал, что Конституция такая хреновая, то почему молчал 20 лет? 
 -представляете ÷ -вася -задумчивым -так .комментарий .в -ты.чё -крест-накрест -и -умирают -привет ..... -международная -вася -одних .есть

>  Если вас всё устраив

In [59]:
# Play with size
wanted_size = 30

rng = np.random.default_rng()
generated = model.decode(torch.from_numpy(rng.random((1, dataset.shape[1]//4))).float())

# Since we use constant-length vectors, we have to cut results
values: np.ndarray = np.round(testing.detach().numpy())[:wanted_size]
print(' '.join([reverse_lookup.get(idx) for idx in values.tolist() if idx in reverse_lookup.keys()]).lower())

-перепись x -хо-хо -в -российским .вчера ..... -одних -извините -дочка -сергей -пишите -просто -колоссально -царем -зря .б .она .вечером -российским -задумчивым .она -и .2022 -божешь .8 -продал -прибыли -хватит -вот
